In [0]:
service_credential = dbutils.secrets.get(scope="ws-scope",key="weather-sp-secret")

spark.conf.set("fs.azure.account.auth.type.jcystorage.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.jcystorage.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.jcystorage.dfs.core.windows.net", "a2210f7a-e661-45c1-98f4-af64e6d2df9f")
spark.conf.set("fs.azure.account.oauth2.client.secret.jcystorage.dfs.core.windows.net", service_credential)
spark.conf.set("fs.azure.account.oauth2.client.endpoint.jcystorage.dfs.core.windows.net", "https://login.microsoftonline.com/a3b377f7-39e7-4ca3-bf66-ea94bbdcdbf3/oauth2/token")

In [0]:
from datetime import datetime, timedelta
from pyspark.sql.functions import col, floor

staging_dir = "abfss://weather-container-staging@jcystorage.dfs.core.windows.net/"
common_dir = "abfss://weather-container-common@jcystorage.dfs.core.windows.net/"

dates_to_process_df = spark.sql("SELECT processed_date FROM processed_dates WHERE process_step = 'staging'")

staging = dbutils.fs.ls(staging_dir)

dates_to_process = [row['processed_date'].strftime('%Y-%m-%d') for row in dates_to_process_df.select(col("processed_date")).collect()]
print(dates_to_process)

try:
    location_enrichment_df = spark.sql("SELECT * FROM location_enrichment")
    location_enrichment_df = location_enrichment_df.withColumn("lat_trunc", floor(col("lat").cast("double") * 100) / 100)
    location_enrichment_df = location_enrichment_df.withColumn("lng_trunc", floor(col("lng").cast("double") * 100) / 100)
    location_enrichment_df = location_enrichment_df.drop("lat", "lng")
except Exception as e:
    print(f"Error obteniendo la tabla de encrichment: {e}")

for date in dates_to_process:
    try:
        weather_staging_df = spark.read.format("delta").load(f"{staging_dir}data/{date}")
        weather_staging_df = weather_staging_df.withColumn("latitud",  floor(col("lat").cast("double") * 100) / 100).withColumn("longitud",  floor(col("lat").cast("double") * 100) / 100)
        enriched_weather_data = enriched_weather_data = weather_staging_df.join(
            location_enrichment_df,
            (weather_staging_df["latitud"] == location_enrichment_df["lat_trunc"]) &
            (weather_staging_df["longitud"] == location_enrichment_df["lat_trunc"]),
            "inner"
)
        display(enriched_weather_data)
        enriched_weather_data.write.format("delta").mode("overwrite").save(f"{common_dir}data/{date}")
    except Exception as e:
        print(f"Error procesando la fehca {date} error: {e}")




